In [7]:
import geopandas
import geoalchemy2
from sqlalchemy import create_engine
import os
import boto3
import json

In [5]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='OMITTED')

In [10]:
secrets = json.loads(response['SecretString'])
host = secrets['host']
password = secrets['password']
username = secrets['username']
database = secrets['dbname']

In [24]:
table_prefix = 'eis_fire'
data_dir = 'data/eis_fire'
years = ['2019', '2020']

In [19]:
files_to_insert = []
for year in years:
    directory = f"{data_dir}/{year}"
    files = os.listdir(directory)
    for file in files:
        files_to_insert.append(f"{directory}/{file}")

files_to_insert

['data/eis_fire/2019/newfirepix.fgb',
 'data/eis_fire/2019/perimeter.fgb',
 'data/eis_fire/2019/fireline.fgb',
 'data/eis_fire/2020/newfirepix.fgb',
 'data/eis_fire/2020/perimeter.fgb',
 'data/eis_fire/2020/fireline.fgb']

In [41]:
file_to_insert = files_to_insert[5]
tablename = f"{table_prefix}_{file_to_insert.split('/')[-1].split('.')[0]}"
print(f"inserting {file_to_insert} into {tablename}")
connection_string = f"'host={host} dbname={database} user={username} password={password}'"
!ogr2ogr -f "PostgreSQL" PG:{connection_string} -t_srs EPSG:4326 {file_to_insert} -nln {tablename} -append

inserting data/eis_fire/2020/fireline.fgb into eis_fire_fireline


In [47]:
# Check the number of rows against the database insertion

In [53]:
tables = ['newfirepix', 'perimeter', 'fireline']
# check number of rows in data frames
for table in tables:
    input_files = [f"{data_dir}/{year}/{table}.fgb" for year in years]
    # count rows in both inputs from 2019 and 2020
    rows = 0
    for file in input_files:
        df = geopandas.read_file(file)
        rows += df.shape[0]
    print(f"Input files for {table} have {rows} rows combined.")
        

Input files for newfirepix have 44488 rows combined.
Input files for perimeter have 44567 rows combined.
Input files for fireline have 43661 rows combined.


In [27]:
# this results in an internal server error when trying to load items in the VEDA Features API
#   File "pydantic/main.py", line 342, in pydantic.main.BaseModel.__init__
#pydantic.error_wrappers.ValidationError: 10 validation errors for FeatureCollection
#features -> 0 -> id
 # str type expected (type=type_error.str)
#features -> 1 -> id
#df_reproj.to_postgis(tablename, engine, index=False, if_exists="append")

In [45]:
# from sqlalchemy import text
# engine = create_engine(f"postgresql://{username}:{password}@{host}:5432/{database}")  
# sql = text('DROP TABLE IF EXISTS fire_boundaries3;')
# result = engine.execute(sql)